# Test Sparkify's S3 to Redshift ETL Notebook

This notebook is used as a sanity check to see if the ETL process worked properly. We will connect to the Redshift cluster and use several queries to make sure everything worked properly.

### Import Libraries

In [15]:
from configparser import ConfigParser
import psycopg2
import psycopg2.extras
import pandas as pd

### Connect to Database

In [17]:
# Get information from configuration and
# connect to Redshift
config = ConfigParser()
config.read_file(open('dwh.cfg'))

conn_string = "host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values())
conn = psycopg2.connect(conn_string)

print(conn_string)

host=redshiftdwh.c0fxig4ucntn.us-west-2.redshift.amazonaws.com dbname=dev user=awsuser password=Redshift0 port=5439


### Test SQL Queries

In [18]:
# https://stackoverflow.com/questions/35604186/convert-psycopg2-dictrow-query-to-pandas-dataframe
# This returns the headers of each tuple/row so we can get the column names
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

In [19]:
query = """
    SELECT page, count(*)
    FROM staging_events
    GROUP BY page
"""

execute_query(cur=cur, query=query) 

,page,count
0,Home,806
1,NextSong,6820
2,About,36
3,Settings,56
4,Login,92
5,Logout,90
6,Upgrade,21
7,Downgrade,60
8,Help,47
9,Save Settings,10


In [20]:
query = """
    SELECT * FROM staging_songs LIMIT 5;
"""

execute_query(cur=cur, query=query) 

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARMAC4T1187FB3FA4C,40.82624,-74.47995,"Morris Plains, NJ",The Dillinger Escape Plan,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,207.77751,2004
1,1,ARANFET1187FB56607,NaN,NaN,"Buff Bay, Portland, Jamaica",Wayne Wonder,SOCGVXG12A6D4FBC3C,All This Time,227.70893,0
2,1,ARXWAQQ1187B9AE954,NaN,NaN,"Newark, NJ",Redman,SORWHWY12A6702038E,WKYA (drop),124.18567,2001
3,1,AR6C8EJ1187FB3F473,NaN,NaN,"Ubá, Minas Gerais",Nelson Ned,SOCCQIQ12A8C13C76B,Brasas Vivas,211.80036,0
4,1,ARTFTSM11C8A415955,NaN,NaN,,Ace Enders & A Million Different People,SORMXGO12AB017CE1D,New Guitar,66.61179,2009


In [21]:
query = """
    SELECT * FROM songplays LIMIT 5
"""

execute_query(cur=cur, query=query) 

,songplay_id,timestamp,song_id,artist_id,user_id,session_id,level,location,user_agent
0,274,1541106673796,SOEIQUY12AF72A086A,ARHUC691187B9AD27F,8,139,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
1,233,1541273619796,SOYTFSY12A6D4FD84E,ARRFSMX1187FB39B03,95,152,paid,"Winston-Salem, NC","""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like..."
2,17,1541440182796,SOHDWWH12A6D4F7F6A,ARC0IOF1187FB3F6E6,73,255,paid,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
3,201,1541440472796,SOWEFTO12A3F1EB976,ARPN0Y61187B9ABAA0,97,147,paid,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
4,20,1541440837796,SOTUWPH12AB017DEBC,ARGLI7X1187B9930BB,97,147,paid,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."


In [24]:
# List artists with the most number plays descending
query = """
SELECT
    a.artist_name,
    a.artist_id,
    count(*) AS count
FROM songplays sp
JOIN artists a 
ON sp.artist_id = a.artist_id
GROUP BY a.artist_id, a.artist_name
ORDER BY count DESC
LIMIT 5
"""

execute_query(cur=cur, query=query) 

,artist_name,artist_id,count
0,Dwight Yoakam,AR5E44Z1187B9A1D74,37
1,Kid Cudi / Kanye West / Common,ARD46C811C8A414F3F,10
2,Ron Carter,AR37SX11187FB3E164,9
3,Lonnie Gordon,AR5EYTL1187B98EDA0,9
4,B.o.B,ARKQQZA12086C116FC,8


In [26]:
# List users by average session duration descending
query = """
SELECT
    avg(sessions.duration) AS mean_duration,
    user_id
FROM (
    SELECT
        max(timestamp) - min(timestamp) AS duration,
        session_id,
        user_id
    FROM songplays
    GROUP BY session_id, user_id
) sessions
GROUP BY user_id
ORDER BY mean_duration DESC
LIMIT 5
"""

execute_query(cur=cur, query=query) 

,mean_duration,user_id
0,9810000,30
1,8204400,24
2,5919000,72
3,5491454,97
4,5082000,15


### Drop All Tables

In [14]:
conn.close()